## Gaussian Elimination

In linear algebra, *Gaussian Elimination* (AKA *row reduction*) is an algorithm for solving **linear systems** of equations $Ax=b$. This method can be used to find the *rank* of a square matrix, calculate the *determinant* and calculate the *inverse* of an invertible square matrix. For example:

$$
\begin{array}{lcl} 
    9x + 3y + 4z & = & 7 \\
    4x + 3y + 4z & = & 8 \\
    x + y + z & = & 3 
\end{array}
$$

can be arranged as:

$$
\mathbf{A}=\left[\begin{array}{ccc|c}
9 & 3 & 4 \\
4 & 3 & 4\\
1 & 1 & 1
\end{array}\right], \qquad 
\mathbf{x} = \left[\begin{array}{ccc|c}
x \\ y \\ z
\end{array}\right], \qquad
\mathbf{b}=\left[\begin{array}{ccc|c}
7 \\ 8 \\ 3
\end{array}\right]
$$

To perform row-reduction, one uses a sequence of **elementary row operations** to modify the matrix until the matrix is in upper-triangular form. There are 3 primary operations:
1. Swapping two rows
2. Multiplying a row by a non-zero number
3. Adding a multiple of one row to another row

e.g: For example consider the matrix

$$
\left[\begin{array}{ccc|c}
9 & 3 & 4 & 7 \\
4 & 3 & 4 & 8 \\
1 & 1 & 1 & 3
\end{array}\right] 
$$

We could begin by swapping the first and third rows, which gives:

$$
\left[\begin{array}{ccc|c}
1 & 1 & 1 & 3 \\
4 & 3 & 4 & 8 \\
9 & 3 & 4 & 7 \\
\end{array}\right]
$$

Next, subtracting 9 times the first row from the third gives:

$$
\left[\begin{array}{ccc|c}
1 & 1 & 1 & 3 \\
4 & 3 & 4 & 8 \\
0 & -6 & -5 & -20 \\
\end{array}\right]
$$

Subtracting 4 times the first row from the second gives:

$$
\left[\begin{array}{ccc|c}
1 & 1 & 1 & 3 \\
0 & -1 & 0 & -4 \\
0 & -6 & -5 & -20 \\
\end{array}\right]
$$

Finally, adding -6 times the second row from the third gives:

$$
\left[\begin{array}{ccc|c}
1 & 1 & 1 & 3 \\
0 & -1 & 0 & -4 \\
0 & 0 & -5 & 4 \\
\end{array}\right]
$$

The final matrix is referred to as *Echelon form*. Note that *Gaussian Elimination* is applied to matrix $A$ and the output vector $b$ simultaenously. Once the matrix is in the final state, we can use another method such as *backward substitution* to solve the linear system trivially.

Here we solve the equation of the $k$th row for $x_k$, then substitute back into the equation of the $(k-1)$st row to obtain a solution for $x_{k-1}$, etc., as:

$$
x_i=\frac{1}{a_{ii}'}\left(b_i' - \sum_{j=i+1}^k a_{ij}'x_j\right)
$$

In [1]:
import numpy as np
A = np.array([[9, 3, 4], [4, 3, 4], [1, 1, 1]], dtype=np.float64)
b = np.array([7, 8, 3], dtype=np.float64)

### Step 1.

It is important to join together $A$ and $b$ at the start of the process: $A + b=A^{*}$

In [4]:
n = len(b)
aug = np.hstack((A, b.reshape(n,1)))
aug

array([[9., 3., 4., 7.],
       [4., 3., 4., 8.],
       [1., 1., 1., 3.]])

### Step 2.

As we can see this joins together $A$ and $b$ into one matrix, ready for elimination. Now we need to loop over the rows, and find the one with the highest *magnitude*, if the current row isn't the maximum, then *swap the rows* until the one with the highest magnitude is closer to the top of the matrix.

In [5]:
max_row = np.argmax(aug[:,0])
max_row

0

### Step 3. Pivoting

If max_row is not 0 (i.e not the top column) then we swap the argmax row with the top one:

In [6]:
if (max_row):
    tmp = np.copy(aug[0,:])
    aug[0,:] = np.copy(aug[max_row,:])
    aug[max_row,:] = tmp

In [7]:
aug

array([[9., 3., 4., 7.],
       [4., 3., 4., 8.],
       [1., 1., 1., 3.]])

### Step 4.

So we have pivoted the top row with the bottom row. Now we want to iterate over all of the rows below our current one (0), and remove the value at the left-most side of the matrix, then scale by the current row:

$$
A^{*}_j = A^{*}_j - \frac{A^{*}_{ji}}{A^{*}_{ii}} A^{*}_i
$$

In [8]:
print("j:{}, {} \n \n".format(0,aug))
for j in range(1, n):
    aug[j,:] = aug[j,:] - (aug[j,0] / aug[0, 0]) * aug[0, :]
    print("j:{}, {} \n \n".format(j,aug))

j:0, [[9. 3. 4. 7.]
 [4. 3. 4. 8.]
 [1. 1. 1. 3.]] 
 

j:1, [[9.         3.         4.         7.        ]
 [0.         1.66666667 2.22222222 4.88888889]
 [1.         1.         1.         3.        ]] 
 

j:2, [[9.         3.         4.         7.        ]
 [0.         1.66666667 2.22222222 4.88888889]
 [0.         0.66666667 0.55555556 2.22222222]] 
 



What we've managed to do is eliminate two of the lower-left hand side terms, making them zero and inducing triangularity. This needs to be repeated for all $i$.

## Tasks

### Task 1.

Write the algorithm `gaussian_elimination()` which accepts $A$ and $b$, and returns the reduced matrix $A^{*}$ and reduced vector $b^{*}$. Use the example at the beginning of this section as input to see if you get the same output. **Note: you shouldn't!**

In [ ]:
# your codes here
def gaussian_elimination(A, b):
    """
    Simple Gaussian Elimination algorithm with pivoting, using direct methods.
    Solves linear system Ax = b, assuming well-conditioned square matrix A.
    
    Parameters
    ----------
    A : matrix
        Square matrix, size n x n.
    b : vector
        RHS vector, size n (must conform with A).

    Returns
    -------
    x : vector
        unknown coefficients, size n
    """
    # Store size of system
    n = len(b)
    assert(np.all(A.shape == (n, n)))
    
    # Form augmented matrix
    aug = np.hstack((A, b.reshape(n,1)));
    
    # Loop over rows
    for i in range(n):
        # Find the row with largest magnitude, and then swap the rows.
        max_row = np.argmax(aug[i:, i])
        
        if (max_row): # Only swap rows if the maximum is not this row. \
                      # NOTE: the max_row is counted relative to i, so max_row = 0 => row i.
            tmp               = np.copy(aug[i, :])
            aug[i, :]         = np.copy(aug[i+max_row, :])
            aug[i+max_row, :] = np.copy(tmp)
        
        # Loop over rows below i
        for j in range(i+1, n):
            aug[j, :] -= (aug[j, i] / aug[i, i] * aug[i, :])
    
    # Return the separated, reduced, matrix and right hand side vector.
    return (aug[:,:-1], aug[:, -1])

A = np.array([[9, 3, 4], [4, 3, 4], [1, 1, 1]], dtype=np.float64)
b = np.array([7, 8, 3], dtype=np.float64)
Ast, bst = gaussian_elimination(A,b)
Ast, bst

### Task 2.

*Backward substitution* can be performed on an upper-triangular matrix $A^{*}$ with known outputs $b^{*}$ to discover the unknown weights $\bf x^{*}$. This is achieved by iterately going up the matrix, substituting known $\bf x^{*}$ values at the previous step into matrix $A^{*}$ and calculating new $\bf x^{*}$. You should expect $x=-1/5$, $y=4$ and $z=-4/5$, within $\bf x^{*}$.

Write the `backward_substitution()` algorithm that determines $x$, $y$, and $z$ within $\bf x$.

In [ ]:
# your codes here
def backward_substitution(Astar, bstar):
    """
    A* = reduced upper-triangular form
    b* = reduced vector
    
    Starts at bottom of matrix and works way up.
    """
    n,p = Astar.shape
    xstar = np.empty(p)
    
    xstar[-1] = bstar[-1] / Astar[-1,-1]
    for i in range(n-2, -1,-1):
        """
        print("i: %d" % i)
        print("x: %s" % xstar[i+1:])
        print("b: %s" % bstar[i])
        print("A: %s,%s" % (Astar[i,i+1:], Astar[i,i]))
        """
        xstar[i] = (bstar[i] - np.dot(xstar[i+1:], Astar[i,i+1:])) / Astar[i,i]
    return xstar

backward_substitution(Ast, bst)